# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part V: Training our End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a simple - but fairly effective - logistic regression model.  More generally, however, Snorkel plugs in with many ML libraries including [TensorFlow](https://www.tensorflow.org/), making it easy to use almost any state-of-the-art model as the end extractor!

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

import numpy as np
from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass, and load the test set:

In [ ]:
from snorkel.models import candidate_subclass
Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

## 1. Automatically Creating Features

First, we create features over the candidates in the training set. These features characterize the text and dependency path information related to the two person mentions in the candidate. **Note that we will define the set of features we use based on the training set here.**  Also note that this operation may take 5-10 minutes, so for large sets, parallelism should be used (by using a database like postgres and setting the `parallelism` keyword argument of `apply`:

In [ ]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

In [ ]:
%time F_train = featurizer.apply(split=0)
F_train

Next, we **apply the feature set we just got from the training set to the dev and test sets** by using `apply_existing`: 

In [ ]:
%%time
F_dev  = featurizer.apply_existing(split=1)
F_test = featurizer.apply_existing(split=2)

If we've already computed the features, again we can just use the below step:

In [ ]:
F_train = featurizer.load_matrix(session, split=0)
F_dev   = featurizer.load_matrix(session, split=1)
F_test  = featurizer.load_matrix(session, split=2)

And, we reload our **noise-aware training labels** (or _training marginals_) from the previous notebook, loading in the same ordering as the feature matrices above:

In [ ]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, F_train, split=0)

## 2. Training the Discriminative Model
We use the training marginals to train a discriminative model that classifies each `Candidate` as a true or false mention. We'll use a random hyperparameter search, evaluated on the development set labels, to find the best hyperparameters for our model. To run a hyperparameter search, we need labels for a development set. If they aren't already available, we can manually create labels using the Viewer.

In [ ]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

Now we set up and run the hyperparameter search, training our model with different hyperparamters and picking the best model configuration to keep. We'll set the random seed to maintain reproducibility.

Note that we are fitting our model's parameters to the training set generated by our labeling functions, while we are picking hyperparamters with respect to score over the development set labels which we created by hand.

In [ ]:
from snorkel.learning.utils import (
    MentionScorer, RandomSearch, ListParameter, RangeParameter
)

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-6, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-6, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-6, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=20)

Next, we'll load in our dev set labels. We will pick the optimal result from the hyperparameter search by testing against these labels:

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)

Finally, we run the hyperparameter search / train the end extraction model:

In [ ]:
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=50, rebalance=0.5, print_freq=25)

_Note that to train a model without tuning any hyperparameters (at your own risk) just use the `train` method of the discriminative model. For instance, to train with 20 epochs and a learning rate of 0.001, you could run:_
```
disc_model.train(F_train, train_marginals, n_epochs=20, lr=0.001)
```

We can analyze the learned model by examining the weights. For example, we can print out the features with the highest weights.

In [ ]:
w, _ = disc_model.get_weights()
largest_idxs = reversed(np.argsort(np.abs(w))[-5:])
for i in largest_idxs:
    print('Feature: {0: <70}Weight: {1:.6f}'.format(F_train.get_key(session, i).name, w[i]))

## 3. Evaluating on the Test Set

In this last section of the tutorial, we'll get the score we've been after: the performance of the extraction model on the blind test set (`split` 2). First, we load the test set labels and gold candidates we made in Part III.

In [ ]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)

Now, we get the precision, recall, and F1 score from the discriminative model:

In [ ]:
p, r, f1 = disc_model.score(F_test, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [ ]:
tp, fp, tn, fn = disc_model.error_analysis(session, F_test, L_gold_test)

Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!